In [ ]:
import pandas as pd
import os

# Parâmetros Estabelecimentos
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

diretorio_empresa = r'base_csv_empresas/'
all_files_estabelecimentos = list(filter(lambda x: '.csv' in x, os.listdir(diretorio_empresa)))

for file_empresa in all_files_estabelecimentos:
    d_empresa = pd.read_csv(f'{diretorio_empresa}{file_empresa}',dtype=dtypes_EMPRESA,error_bad_lines=False,sep=';', nrows=10)
    print(d_empresa)

In [1]:
# libs
import pandas as pd
import json
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

dados = pd.read_csv('DB_SHOW_APPLE.csv')
dados.head(2)

,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,TELEFONE,EMAIL,CNAE_PRINCIPAL,CNAE_DESCRICAO
0,58541000170,Q COISA BOA BAR LTDA,BUTECO TOMAUMA,RUA MATEUS LEME,2631,NaN,BOM RETIRO,CURITIBA,PR,80520174.0,(41)98713461,DELIMA.ADV@GMAIL.COM,5611205,Restaurantes e similares
1,108202000151,ARMAZEM COQUEIRO LTDA,ARMAZEM COQUEIRO,ESTRADA DO CABRAL,0,NaN,CABRAL,PARATY,RJ,23970000.0,(24)337127,VITRINA@VITRINACONTABILIDADE.COM.BR,5611205,Restaurantes e similares


In [2]:
#dados['COORDENADAS'] = dados['CEP'].apply(lambda x: geolocator.geocode(x, timeout=10))
dados['ENDERECO'] = dados['BAIRRO'].map(str) + ", " + dados['CIDADE'].map(str) + ", " + dados['ESTADO'].map(str)
dados['COORDENADAS'] = dados['ENDERECO'].apply(lambda x: geolocator.geocode(x, timeout=10))
dados['COORDENADAS']

0      (Bom Retiro, Matinhos, Região Geográfica Imedi...
1      (Rua Pedro Álvares Cabral, Paraty, Região Geog...
2      (Centro, Nova Veneza, Região Geográfica Imedia...
3      (Centro Comercial Bom Princípio, 430, ERS-122,...
4      (Três Barras, Linhares, Região Geográfica Imed...
                             ...                        
495    (Centro, Caraguatatuba, Região Imediata de Car...
496    (Serra, Regional Centro-Sul, Belo Horizonte, R...
497    (Vila Esteves, Limeira, Região Imediata de Lim...
498                                                 None
499    (Centro, Itabuna, Região Geográfica Imediata d...
Name: COORDENADAS, Length: 500, dtype: object

In [3]:

dados['LATITUDE'] = dados.apply(lambda row: row['COORDENADAS'].latitude if row['COORDENADAS'] is not None else None, axis=1)
dados['LONGITUDE']  = dados.apply(lambda row: row['COORDENADAS'].longitude if row['COORDENADAS'] is not None else None, axis=1)


In [4]:
dados = dados.sort_values(by='TELEFONE', ascending=True)

In [49]:
import random
aleatorio = random.randint(0, 3)
horario = ["""domingo 
Fechado

segunda-feira
11:00–15:00
18:00–23:00

terça-feira
11:00–15:00
18:00–23:00

quarta-feira
11:00–15:00
18:00–23:00

quinta-feira
11:00–15:00
18:00–23:00

sexta-feira
11:00–15:00
18:00–23:00

sábado
11:00–15:00
18:00–23:00""",
"""domingo
Fechado

segunda-feira
11:00–15:00

terça-feira
11:00–15:00

quarta-feira
11:00–15:00

quinta-feira
11:00–15:00

sexta-feira
11:00–15:00

sábado
Fechado""",
"""
domingo
Fechado

segunda-feira
10:00–14:30
18:00–00:00

terça-feira
10:00–14:30
18:00–00:00

quarta-feira
10:00–14:30
18:00–00:00

quinta-feira
10:00–14:30
18:00–00:00

sexta-feira
10:00–14:30
18:00–00:00

sábado
10:00–14:30
18:00–00:00""",
"""domingo
Fechado

segunda-feira
11:00–15:30

terça-feira
11:00–15:30

quarta-feira
11:00–15:30

quinta-feira
11:00–15:30

sexta-feira
11:00–15:30

sábado
11:00–15:00"""]
import time
for cnpj in range(0,len(dados['CNPJ'])-1 ):
    dados.at[cnpj, 'HORARIO_FUNCIONAMENTO'] = horario[aleatorio]
    time.sleep(0.1)
dados



,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,RUA,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,ESTADO,CEP,...,CNAE_DESCRICAO,ENDERECO,COORDENADAS,LATITUDE,LONGITUDE,HORARIO_FUNCIONAMENTO,SITE,FACEBOOK,INSTAGRAM,OPCOES_DE_SERVICO
129,23030000113,GILMAR A FRARE,PADARIA MORUNGABA,PRAÇA JOAO PESSOA,142,NaN,CENTRO,MORUNGABA,SP,13260000.0,...,Lanchonetes casas de chá de sucos e similares,"CENTRO, MORUNGABA, SP","(Sítio São Damião, Estrada Municipal Um, Morun...",-22.832452,-46.800462,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOPADARIAMORUNGABA.COM,https://pt-br.facebook.com/padariamorungaba,@padariamorungaba,\nRetirada na porta\nEntrega sem contato\nEntr...
391,15700000150,F. C. F. PESQUEIRO E RESTAURANTE LTDA,PESQUEIRO MURAYAMA,AVENIDA P.L. DO BRASIL,SM,KM 05,CENTRO,ARUJA,SP,7400630.0,...,Bares e outros estabelecimentos especializados...,"CENTRO, ARUJA, SP","(Centro, Arujá, Região Imediata de São Paulo, ...",-23.394437,-46.320446,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOPESQUEIROMURAYAMA.COM,https://pt-br.facebook.com/pesqueiromurayama,@pesqueiromurayama,\nRetirada na porta\nEntrega sem contato\nEntr...
36,1165656000127,BAR NASCIMENTO E SOUZA LTDA,Indisponível,RUA PRES. GETULIO DORNELES VARGAS,187,CASA 01,VILA NOVA MAIRINQUE,MAIRINQUE,SP,18120000.0,...,Restaurantes e similares,"VILA NOVA MAIRINQUE, MAIRINQUE, SP","(Rodovia Raposo Tavares, Vila Sorocabana, Mair...",-23.538229,-47.184989,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOIndisponível.COM,https://pt-br.facebook.com/indisponível,@indisponível,\nRetirada na porta\nEntrega sem contato\nEntr...
205,996000135,LL LANCHONETE LTDA,Indisponível,RUA VOLUNTARIOS DA PATRIA,512,TERREO520,SANTANA,SAO PAULO,SP,2010000.0,...,Bares e outros estabelecimentos especializados...,"SANTANA, SAO PAULO, SP","(Herculândia, Região Imediata de Tupã, Região ...",-22.004508,-50.386275,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOIndisponível.COM,https://pt-br.facebook.com/indisponível,@indisponível,\nRetirada na porta\nEntrega sem contato\nEntr...
181,116504000171,WERDEN CONSULTORIA EMPRESARIAL E PARTICIPACOES...,WERDEN BAR,RUA AMAURI,284,NaN,ITAIM BIBI,SAO PAULO,SP,1448000.0,...,Lanchonetes casas de chá de sucos e similares,"ITAIM BIBI, SAO PAULO, SP","(Itaim Bibi, São Paulo, Região Imediata de São...",-23.584381,-46.678444,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOWERDENBAR.COM,https://pt-br.facebook.com/werdenbar,@werdenbar,\nRetirada na porta\nEntrega sem contato\nEntr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,4897000121,MAURO AP.GABRIEL,Indisponível,RUA CUBA,218,NaN,VILA DIAS,MOGI MIRIM,SP,13802085.0,...,Bares e outros estabelecimentos especializados...,"VILA DIAS, MOGI MIRIM, SP",None,NaN,NaN,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOIndisponível.COM,https://pt-br.facebook.com/indisponível,@indisponível,\nRetirada na porta\nEntrega sem contato\nEntr...
276,4835000110,CECILIA YOSHIE TANABE YAMAGUCHI,Indisponível,AVENIDA JOSE DE A CARVALHO,1800,NaN,VILA JUDITH,ITAPETININGA,SP,18206420.0,...,Bares e outros estabelecimentos especializados...,"VILA JUDITH, ITAPETININGA, SP","(Vila Judith, Itapetininga, Região Imediata de...",-23.577351,-48.030801,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOIndisponível.COM,https://pt-br.facebook.com/indisponível,@indisponível,\nRetirada na porta\nEntrega sem contato\nEntr...
275,4815000149,LANCHONETE E SORVETERIA TYPO'RIKOS LTDA,TYPO'RIKOS,RUA 10 DE ABRIL,53,NaN,CENTRO,JUQUIA,SP,11800000.0,...,Bares e outros estabelecimentos especializados...,"CENTRO, JUQUIA, SP","(Centro de Evento Cecoseba, Rua Juquiá, Jardim...",-24.386115,-47.925233,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,EXEMPLOTYPO'RIKOS.COM,https://pt-br.facebook.com/typo'rikos,@typo'rikos,\nRetirada na porta\nEntrega sem contato\nEntr...
282,5129000192,BAR E LANCHONETE LAREDO LTDA,Indisponível,RUA DA CORA,1441,NaN,VILA GUILHERME,SAO PAULO,SP,2047020.0,...,Bares e outros estabeleci

In [6]:
dados['SITE'] = "EXEMPLO" + dados['NOME_FANTASIA'].str.replace(" ", "", regex=True) + ".COM"
dados['FACEBOOK'] = "https://pt-br.facebook.com/" + dados['NOME_FANTASIA'].str.lower().replace(" ","",regex=True)
dados['INSTAGRAM'] = "@"+ dados['NOME_FANTASIA'].str.lower().replace(" ","", regex=True)
opcoes_servicos = [
    """
Entrega sem contato
Entrega
Para viagem
Refeição no local
    """,
    """
Para viagem
Refeição no local
    """,
    """
Mesas externas
Entrega sem contato
Entrega
Para viagem
Refeição no local
    """,
    """
Retirada na porta
Entrega sem contato
Entrega
Para viagem
Refeição no local
    """
]
import time
aleatorio = random.randint(0, 3)
for cnpj in range(0,len(dados['CNPJ'])-1 ):
    dados.at[cnpj, 'OPCOES_DE_SERVICO'] = opcoes_servicos[aleatorio]
    time.sleep(0.1)
dados['OPCOES_DE_SERVICO'].head(2)

In [50]:
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE', 'TELEFONE', 'SITE',
       'CNAE_DESCRICAO', 'HORARIO_FUNCIONAMENTO', 'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO']]


In [51]:
indice_remove = dados[(dados['TELEFONE'] =="(nan)nan")].index
dados.drop(indice_remove, inplace=True)
len(dados)


C:\Users\ABRASEL NACIONAL\AppData\Local\Temp\ipykernel_6832\3448654424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.drop(indice_remove, inplace=True)


214

In [52]:
data = dados.copy()
data.rename(columns={'CNPJ':"EIN (CNPJ)", 'RAZAO_SOCIAL':'CORPORATE_NAME', 'NOME_FANTASIA':'TRADING NAME', 'RUA':'STREET', 'NUMERO':'ADDRESS_NUMBER', 'COMPLEMENTO':'ADDRESS_COMPLEMENT',
       'BAIRRO':'DISTRICT', 'CIDADE':'CITY', 'ESTADO':'STATE', 'CEP':'ZIP_CODE', 'TELEFONE':'PHONE_NUMBER', 'EMAIL':'E-MAIL_ADDRESS','HORARIO_FUNCIONAMENTO':'OPENING_HOURS', 'OPCOES_DE_SERVICO':'SERVICE_OPTIONS', 'CNAE_DESCRICAO':'CATEGORY_(CNAE)'}, inplace=True)


In [53]:
dados.to_csv('BD_AMOSTRA_APPLE.csv', index=False)
data.to_csv('DB_SHOW_APPLE.csv', index=False)

In [11]:
# English version
data.to_json("en_amostra_json/EN_db_SIGA_05_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PHONE_NUMBER', 'SITE','CATEGORY_(CNAE)']]
data.to_json("en_amostra_json/EN_db_SIGA_04_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE', 'LATITUDE', 'LONGITUDE','PHONE_NUMBER', 'SITE']]  
data.to_json("en_amostra_json/EN_db_SIGA_03_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE', 'LATITUDE', 'LONGITUDE']]
data.to_json("en_amostra_json/EN_db_PUBLIC_BASE_02_APPLE.json", orient="records")
data = data[['EIN (CNPJ)', 'CORPORATE_NAME', 'TRADING NAME', 'STREET',
       'ADDRESS_NUMBER', 'ADDRESS_COMPLEMENT', 'DISTRICT', 'CITY', 'STATE',
       'ZIP_CODE']]
data.to_json("en_amostra_json/EN_db_PUBLIC_BASE_01_APPLE.json", orient="records")

# Versão português
dados.to_json("amostra_json/PT_bd_SIGA_05_APPLE.json", orient='records')
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE', 'TELEFONE', 'SITE',
       'CNAE_DESCRICAO']]
dados.to_json("amostra_json/PT_bd_SIGA_04_APPLE.json", orient='records')
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE', 'TELEFONE', 'SITE']]
dados.to_json("amostra_json/PT_bd_SIGA_03_APPLE.json", orient='records')
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP','LATITUDE', 'LONGITUDE']]
dados.to_json("amostra_json/PT_bd_BASE_PUBLICA_02_APPLE.json", orient="records")    
dados = dados[['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP']]
dados.to_json("amostra_json/PT_bd_BASE_PUBLICA_01_APPLE.json", orient="records")